# Time Series Analysis of Ethereum (ETH/USDT) Using ARIMA
Objective: Forecast Ethereum price trends using ARIMA on historical data.

In [ ]:
!pip install yfinance statsmodels pmdarima matplotlib seaborn

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load ETH data
eth = yf.download('ETH-USD', start='2020-01-01', end='2024-12-31')
eth = eth[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
eth.index = pd.to_datetime(eth.index)
eth = eth.asfreq('D').fillna(method='ffill')
eth.head()

In [ ]:
# EDA
eth['Close'].plot(figsize=(14, 6), title='Ethereum Closing Price')
plt.show()
eth['Rolling_Mean'] = eth['Close'].rolling(window=30).mean()
eth[['Close', 'Rolling_Mean']].plot(figsize=(14, 6), title='30-Day Rolling Mean')
plt.show()
eth['Volume'].plot(figsize=(14, 6), title='Ethereum Trading Volume')
plt.show()
eth.describe()

In [ ]:
# Stationarity Test
def adf_test(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    if result[1] <= 0.05:
        print("The data is stationary.")
    else:
        print("The data is non-stationary.")
adf_test(eth['Close'])
eth['Close_diff'] = eth['Close'].diff()
adf_test(eth['Close_diff'].dropna())

In [ ]:
# ACF and PACF
plot_acf(eth['Close_diff'].dropna(), lags=30)
plot_pacf(eth['Close_diff'].dropna(), lags=30)
plt.show()

In [ ]:
# Fit ARIMA
model = ARIMA(eth['Close'], order=(1, 1, 1))
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
# Train-Test Split and Evaluation
train = eth['Close'][:-30]
test = eth['Close'][-30:]
model = ARIMA(train, order=(1, 1, 1))
model_fit = model.fit()
forecast = model_fit.forecast(steps=30)
rmse = np.sqrt(mean_squared_error(test, forecast))
mape = mean_absolute_percentage_error(test, forecast) * 100
print(f'RMSE: {rmse}, MAPE: {mape}%')
plt.figure(figsize=(14, 6))
plt.plot(test.index, test, label='Actual')
plt.plot(test.index, forecast, label='Forecast')
plt.legend()
plt.title('Actual vs Forecast')
plt.show()

In [ ]:
# Forecast Next 30 Days
final_model = ARIMA(eth['Close'], order=(1, 1, 1)).fit()
forecast = final_model.get_forecast(steps=30)
forecast_df = forecast.summary_frame()
plt.figure(figsize=(14, 6))
plt.plot(eth['Close'], label='Historical')
plt.plot(forecast_df['mean'], label='Forecast', color='green')
plt.fill_between(forecast_df.index, forecast_df['mean_ci_lower'], forecast_df['mean_ci_upper'], color='green', alpha=0.2)
plt.legend()
plt.title('Ethereum Price Forecast for Next 30 Days')
plt.show()